In [1]:
import torchxrayvision as xrv
import skimage, torch, torchvision
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2 as transforms
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import cv2
from skimage import io, transform
import matplotlib.patches as patches
import random
import sys
import os
import re
from data_aug.data_aug import *
from data_aug.bbox_util import *
from torchsummary import summary
from torchviz import make_dot
from tqdm import tqdm
import csv
import matplotlib.pyplot as plt

In [2]:
def Vs(ruta=None,matrix=None):
  if ruta:
    image = io.imread(ruta)
    plt.imshow(image,cmap='gray')
  else:
    image = matrix
    plt.imshow(image,cmap='gray')

In [3]:
def Visualize(img,bboxes,bb=False,ruta=False,dim=False,bz=1,bboxtipe='coco'):

    if ruta:
      print('ruta:')
      print(r)
    if bb:
      print('bbox:')
      print(bboxes)
    if dim:
      print('img inf:')
      print(type(img))

    for i in range(bz):
      if bboxtipe=='coco':
          bboxes=[bboxes[0].item(),bboxes[1].item(),bboxes[2].item(),bboxes[3].item()]
      else:
          bboxes=[bboxes[0].item(),bboxes[1].item(),bboxes[2].item()-bboxes[0].item(),bboxes[3].item()-bboxes[1].item()]
      fig, ax = plt.subplots(figsize=(18, 5), facecolor='w', edgecolor='b')
      ax.imshow(img[i],cmap='gray')
      rect = patches.Rectangle((bboxes[0],bboxes[1]),bboxes[2],bboxes[3], linewidth=1.5, edgecolor='r', facecolor='none')
      ax.add_patch(rect)

      plt.show()

In [4]:
def VisualizeINT(img,bboxes,bb=False,ruta=False,dim=False,bz=1,bboxtipe='coco'):

    if ruta:
      print('ruta:')
      print(r)
    if bb:
      print('bbox:')
      print(bboxes)
    if dim:
      print('img inf:')
      print(img.shape)

    for i in range(bz):
      if bboxtipe=='coco':
          bboxes=[bboxes[0].item(),bboxes[1].item(),bboxes[2].item(),bboxes[3].item()]
      else:
          bboxes=[bboxes[0].item(),bboxes[1].item(),bboxes[2].item()-bboxes[0].item(),bboxes[3].item()-bboxes[1].item()]
      fig, ax = plt.subplots(figsize=(18, 5), facecolor='w', edgecolor='b')
      ax.imshow(img[i],cmap='gray')
      rect = patches.Rectangle((int(bboxes[0]),int(bboxes[1])),int(bboxes[2]),int(bboxes[3]), linewidth=1.5, edgecolor='r', facecolor='none')
      ax.add_patch(rect)
      plt.show()

In [5]:
class CustomDataset(Dataset):
    def __init__(self, csv_path, target_size, normalizacion="[-1,1]", type_resize='interpolation', augmentation=None,bboxtipe='coco',dataroot=''):
        self.data = pd.read_csv(csv_path)
        self.norm=normalizacion
        self.target_size = target_size
        self.type_resize = type_resize
        self.augmentation = augmentation
        self.bboxtipe = bboxtipe
        self.dataroot = dataroot

    def __len__(self):
        return len(self.data)

    def resize_image(self, image):
        if  self.type_resize == 'interpolation':
          resized_image=skimage.transform.resize(image, self.target_size)
        else:
          original_height, original_width = image.shape
          target_height, target_width = self.target_size

          # Calcula las escalas de cambio de tamaño en ambas dimensisones
          scale_height = target_height / original_height
          scale_width = target_width / original_width

          # Elige la escala más pequeña para asegurarte de que la imagen se ajuste en el nuevo tamaño
          scale = min(scale_height, scale_width)

          # Calcula las nuevas dimensiones después del cambio de tamaño
          new_height = int(original_height * scale)
          new_width = int(original_width * scale)

          # Realiza el cambio de tamaño utilizando el método 'letterboxing'
          resized_image = np.zeros((target_height, target_width))
          top = (target_height - new_height) // 2
          left = (target_width - new_width) // 2
          resized_image[top:top + new_height, left:left + new_width] = cv2.resize(image, (new_width, new_height))

        return resized_image

    def resize_bbox(self, bbox, original_shape):
        # Redimensionar las coordenadas del bounding box para que se ajusten al nuevo tamañofrom tqdm import tqdm
        x, y, width, height = bbox
        new_width = width * self.target_size[1] / original_shape[1]
        new_height = height * self.target_size[0] / original_shape[0]
        new_x = x * self.target_size[1] / original_shape[1]
        new_y = y * self.target_size[0] / original_shape[0]

        return [new_x, new_y, new_width, new_height]

    def apply_aug(self,image,bbox):
        if self.augmentation:
            p=random.uniform(0.2, 0.6)
            n2=random.randint(3, 9)
            bboxr=np.array([[bbox[0],bbox[1],bbox[0]+bbox[2],bbox[1]+bbox[3]]])
            imgr = np.stack((image, image, image)).transpose(1, 2, 0)
            imgr, bboxr=RandomHorizontalFlip(p)(imgr, bboxr)
            imgr, bboxr=RandomScale(0.1, diff = True)(imgr,bboxr)
            imgr, bboxr=RandomTranslate(0.1, diff = True)(imgr, bboxr)
            imgr, bboxr=RandomRotate(n2)(imgr, bboxr)
            imgr, bboxr=RandomShear(0.1)(imgr, bboxr)
            img=np.array(imgr[...,0]).astype(np.uint8)
            bbox=[float(bboxr[0][0]),float(bboxr[0][1]),float(bboxr[0][2]-bboxr[0][0]),float(bboxr[0][3]-bboxr[0][1])]
        else:
            img=image
            bbox=bbox
        return img , bbox


    def __getitem__(self, idx):
        img_ruta = self.dataroot+self.data['ruta'][idx]
        image = io.imread(img_ruta)
        # Extract bounding box coordinates
        x, y, width, height = self.data.iloc[idx, 1:5]

        image,bbox=self.apply_aug(image,[x,y,width,height])

        if self.norm == "[-1,1]":
          #normaliza a [-1,1]
          if image.dtype == np.uint8:  # Si la imagen es de 8 bits
              image = (image / 255.0) * 2.0 - 1.0
          elif image.dtype == np.uint16:  # Si la imagen es de 16 bits
              image = (image / 65535.0) * 2.0 - 1.0
          else:
              raise ValueError("Tipo de datos de imagen no compatible.")
        else:
          #Normaliza a [0,1]
          if image.dtype == np.uint8:  # Si la imagen es de 8 bits
            image = image / 255.0
          elif image.dtype == np.uint16:  # Si la imagen es de 16 bits
            image = image / 65535.0
          else:
              raise ValueError("Tipo de datos de imagen no compatible.")

        # Redimensionar la imagen y el bounding box
        image_r = self.resize_image(image)
        bbox_r = self.resize_bbox(bbox, image.shape)
        image_r= image_r[np.newaxis, ...]
        if self.bboxtipe != 'coco':
          bbox_r=[bbox_r[0],bbox_r[1],bbox_r[0]+bbox_r[2],bbox_r[1]+bbox_r[3]]
        image_r = torch.tensor(image_r, dtype=torch.float32)
        bbox_r = torch.tensor(bbox_r, dtype=torch.float32)
        return image_r, bbox_r

In [6]:
dataroot = "/home/jair/"

In [7]:
csv_path_traind = dataroot+"Anotaciones_estructuradas/deteccion_train.csv"
csv_path_testd = dataroot+"Anotaciones_estructuradas/deteccion_test.csv"

In [8]:
new_size=(300,300)
normalize="[0,1]"
batch_size=10 #5
data_aug=True
type_resize="interpolation" # letterboxing or interpolation
trans=True
bboxtipe='xy' #coco or xy 

In [9]:
set_traind=CustomDataset(csv_path_traind,target_size=new_size,normalizacion=normalize,type_resize=type_resize,augmentation=trans,bboxtipe=bboxtipe,dataroot=dataroot)
dataloader_traind_t = DataLoader(set_traind, batch_size=batch_size, shuffle=True)
#to execute transformations bboxtipe must be coco
#for batch in dataloader_traind_t:
#    imagenes, bboxes = batch
#    Visualize(imagenes[0],bboxes[0],bb=False,ruta=False,dim=False,bz=batch_size,bboxtipe=bboxtipe) #jala con batch=1
#    break

In [10]:
#originales
set_traind=CustomDataset(csv_path_traind,target_size=new_size,normalizacion=normalize,type_resize=type_resize,augmentation=None,bboxtipe=bboxtipe,dataroot=dataroot)
dataloader_traind = DataLoader(set_traind, batch_size=batch_size, shuffle=True)
#for batch in dataloader_traind:
#    imagenes, bboxes = batch
#    Visualize(imagenes[0],bboxes[0],bb=False,ruta=False,dim=False,bz=batch_size,bboxtipe=bboxtipe) #jala con batch=1
#    break

In [11]:
set_testd=CustomDataset(csv_path_testd,target_size=new_size,normalizacion=normalize,type_resize=type_resize,augmentation=None,bboxtipe=bboxtipe,dataroot=dataroot)
dataloader_testd = DataLoader(set_testd, batch_size=batch_size, shuffle=True)
#for batch in dataloader_testd:
#    imagenes, bboxes = batch
#    Visualize(imagenes[0],bboxes[0],bb=False,ruta=False,dim=False,bz=batch_size,bboxtipe=bboxtipe) #jala con batch=1
#    break

In [12]:
!nvidia-smi

Tue Sep 12 02:43:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 52%   70C    P0   136W / 420W |      0MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   

In [13]:
train_csv = pd.DataFrame(columns=['epoch', 'loss'])
train_csv.to_csv('Modelos/Retinanet_Resnet50/train.csv', index=False)

train_csv = pd.DataFrame(columns=['epoch', 'IoU_average'])
train_csv.to_csv('Modelos/Retinanet_Resnet50/validation.csv', index=False)

In [14]:
def calcular_area(recuadro):
    area=0
    if len(recuadro) >0:
        x_min, y_min, x_max, y_max = recuadro
        area=max(0, x_max - x_min) * max(0, y_max - y_min)    
    return area

def calcular_iou(recuadro1, recuadro2):
    # Calcula las áreas de los recuadros
    
    area1 = calcular_area(recuadro1)
    area2 = calcular_area(recuadro2)
    iou=0
    if area1 >0 and area2 >0:
        # Calcula la intersección entre los recuadros
        x_min = max(recuadro1[0], recuadro2[0])
        y_min = max(recuadro1[1], recuadro2[1])
        x_max = min(recuadro1[2], recuadro2[2])
        y_max = min(recuadro1[3], recuadro2[3])

        # Verifica si hay intersección entre los recuadros
        if x_min >= x_max or y_min >= y_max:
            return 0.0  # No hay intersección

        # Calcula el área de intersección
        area_interseccion = calcular_area([x_min, y_min, x_max, y_max])

        # Calcula el IoU
        iou = area_interseccion / (area1 + area2 - area_interseccion)
    return iou

In [15]:
from sklearn.metrics import average_precision_score
import torchvision.models.detection as md
from torchvision.models.detection import RetinaNet
import torch.nn as nn

tipo = "gpus" if torch.cuda.is_available() else "cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("There is gpu available:", torch.cuda.is_available())
print("The number of gpu's availables:", torch.cuda.device_count())

model = md.retinanet_resnet50_fpn_v2(num_classes=1)

# Verificar si hay múltiples GPUs disponibles y usar DataParallel
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

model.to(device)

# Definir un optimizador, por ejemplo, Adam
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento
num_epochs = 30  # Número de épocas que deseas entrenar

train_num=len(dataloader_traind)
test_num=len(dataloader_testd)

pbar_train = tqdm(total=train_num, unit="iter", desc="Lotes training")
pbar_test = tqdm(total=test_num, unit="iter", desc="Lotes evaluation")

print('Starting training with: ', tipo)
for epoch in tqdm(range(num_epochs)):
    model.train()

    
    loss_values = []  # Lista para guardar los valores de pérdida en cada época

    for images, bboxes in dataloader_traind:
        optimizer.zero_grad()

        images = list(image.to(device) for image in images)

        targets = []
        for bbox in bboxes:
            targets.append({'boxes': torch.tensor([bbox.tolist()]).to(device), 'labels': torch.tensor([0]).to(device)})

        loss_dict = model(images, targets)
        loss_bbox_regression = loss_dict['bbox_regression']
        loss_bbox_regression.backward()
        optimizer.step()
        loss_values.append(loss_bbox_regression.item())
        pbar_train.update(1)
    
    average_loss = np.mean(loss_values)
    print(f'Epoch [{epoch + 1}/{num_epochs}] Average Loss: {average_loss}')

    # Open existing CSV in append mode 'a'
    with open('Modelos/Retinanet_Resnet50/train.csv', mode='a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        # Write the epoch and avetage loss in the CSV
        csvwriter.writerow([epoch + 1, average_loss])
    
    model.eval()

    all_predicted_bboxes=[]
    IOU_average_epoch=0
    for images, bboxes in dataloader_testd:
        with torch.no_grad():
            images = list(image.to(device) for image in images)
            targets = []
            for bbox in bboxes:
                targets.append({'boxes': torch.tensor([bbox.tolist()]).to(device), 'labels': torch.tensor([0]).to(device)})

            predictions = model(images)
            true_bboxes=bboxes.tolist()

            for prediction in predictions:
                all_predicted_bboxes.append(prediction['boxes'].tolist())
            pbar_test.update(1)

            #calculte de iou with all the bboxes and take de maximun (the most closer to the original)
            #sum the max and promediate the final iou for all the lote 
            ioum=0
            for obbox , pbboxes in zip(true_bboxes,all_predicted_bboxes):
                if len(pbboxes) >0:
                    m=0
                    for pbbox in pbboxes:
                        iou=calcular_iou(obbox,pbbox)
                        m=max(m,iou)
                    ioum=ioum+m
            #calculate de mean of the iou per lote
            iou_average_lote=ioum/len(true_bboxes)
            IOU_average_epoch=IOU_average_epoch+iou_average_lote
    #calculate de mean of the iou per epoch        
    IOU_average_epoch=IOU_average_epoch/len(dataloader_testd)
    print(f'Epoch [{epoch + 1}/{num_epochs}] Average IoU: {IOU_average_epoch}')
            
    

    with open('Modelos/Retinanet_Resnet50/validation.csv', mode='a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
         #Write the epoch and avetage loss in the CSV
        csvwriter.writerow([epoch + 1, IOU_average_epoch])
pbar_train.close()        
pbar_test.close()
# Guardar el modelo entrenado al final del entrenamiento
torch.save(model.state_dict(), 'Modelos/Retinanet_Resnet50/modelo_entrenado_originales.pth')

There is gpu available: True
The number of gpu's availables: 2


Lotes evaluation:   0%|                               | 0/158 [00:00<?, ?iter/s]

Starting training with:  gpus




Lotes training: 100%|███████████████████████| 550/550 [35:49<00:00,  3.65s/iter]

Epoch [1/30] Average Loss: 0.4266386625983498


...

Epoch [29/30] Average IoU: 0.13053763259275133


Lotes training: 16500iter [22:24:19,  3.66s/iter] 

Epoch [30/30] Average Loss: 0.2968690512668003



Lotes evaluation: 4583iter [22:24:24, 648.58s/iter]
Lotes evaluation: 4584iter [22:24:30, 455.65s/iter]
Lotes evaluation: 4585iter [22:24:33, 319.77s/iter]
Lotes evaluation: 4586iter [22:24:37, 225.07s/iter]
Lotes evaluation: 4587iter [22:24:40, 158.54s/iter]
Lotes evaluation: 4588iter [22:24:44, 112.05s/iter]
Lotes evaluation: 4589iter [22:24:47, 79.51s/iter] 
Lotes evaluation: 4590iter [22:24:52, 57.14s/iter]
Lotes evaluation: 4591iter [22:24:55, 41.04s/iter]
Lotes evaluation: 4592iter [22:24:58, 29.45s/iter]
Lotes evaluation: 4593iter [22:25:02, 21.93s/iter]
Lotes evaluation: 4594iter [22:25:07, 16.66s/iter]
Lotes evaluation: 4595iter [22:25:10, 12.59s/iter]
Lotes evaluation: 4596iter [22:25:14, 10.11s/iter]
Lotes evaluation: 4597iter [22:25:18,  8.26s/iter]
Lotes evaluation: 4598iter [22:25:21,  6.79s/iter]
Lotes evaluation: 4599iter [22:25:24,  5.68s/iter]
Lotes evaluation: 4600iter [22:25:28,  5.07s/iter]
Lotes evaluation: 4601iter [22:25:31,  4.48s/iter]
Lotes evaluation: 4602i

Epoch [30/30] Average IoU: 0.13565233226934997


In [5]:
train = pd.read_csv('Modelos/Retinanet_Resnet50/train.csv')
val = pd.read_csv('Modelos/Retinanet_Resnet50/validation.csv')

x1 = train['epoch']
y1 = train['loss']

x2 = val['epoch']
y2 = val['IoU_average']

In [6]:
plt.plot(x1, y1, marker='o', linestyle='-') 
plt.title('train graph')
plt.xlabel('epoch')
plt.ylabel('average loss')
plt.savefig('Modelos/Retinanet_Resnet50/train.png')
plt.close()

In [7]:
plt.plot(x2, y2, marker='o', linestyle='-') 
plt.title('validation graph')
plt.xlabel('epoch')
plt.ylabel('average IoU')
plt.savefig('Modelos/Retinanet_Resnet50/validation.png')
plt.close()